In [1]:
#from dataclasses import dataclass, field
import copy
import os
from typing import List, Dict, Union
#from decouple import AutoConfig, config, UndefinedValueError
from dotenv import *
import httpx
from openpyxl import load_workbook, Workbook

#from gbp import proveedor_del_sku, gbp_get_sku_publis
from recolectar_tipos_envio_ml import (ml_aut, Tienda, get_items_ids, Costos_Envio, Precios, \
                                       Round_Costos_Envio, Proveedor, Publicacion, SHIPMENT_TYPES_CATALOG, \
                                        Coeficiente_Envio, get_publis_gbp, get_proveedor_sku)


In [2]:
### ACTUALIZADOR COSTO DE ENVIO PUBLICACIONES EN GBP
#   CONFIGURAR QUË PROVEEDORES ACTUALIZAN COSTO DE ENVIO

#   DETECTAR CAMBIOS EN LOS COSTOS DE ENVIO PROMEDIO DE ML

#   GENERAR EXCEL CON COSTO PROMEDIO DE ENVIO ML PARA ACTUALIZAR EN GBP
    #   ESTE EXCEL TIENE QUE TENER DOS CAMPOS: ID_PUBLICACION y COSTO DE ENVIO

# Traigo costo de envío promedio con esta función de ml.py: costos_envio_colecta_publi(item_id = None, token = None, item_json = None)

In [3]:
#   DETECTAR CAMBIOS EN LOS COSTOS DE ENVIO PROMEDIO DE ML
    #   TRAER PUBLIS ML
        #   FILTRAR POR PROVEEDORES CON ENVIO GRATIS
        #   TRAER COSTO DE ENVIO ML
    #   ABRIR EXCEL PUBLIS_GBP
        #   CALCULAR COSTO DE ENVIO GBP DE PUBLIS ML (en base al precio GBP y ENVIO ML)
        #   COMPARAR COSTOS DE ENVIO EXCEL GBP vs GBP CALCULADO
    #   ACTUALIZAR LOS COSTOS DE ENVIO QUE HAGAN FALTA

In [4]:
#   CONFIGURA QUÉ PROVEEDORES TIENEN COSTO CON ENVIO EN GBP
ALIAS_PROVEEDORES_CON_COSTO_ENVIO = ["microglobal", "bowie"]

#   CONFIGURA QUÉ TIENDAS CONTROLA
LISTA_TIENDAS = ["tecnorium", "lenovo"]

#   CONFIGURA ARCHIVO CON PUBLIS_GBP
PUBLIS_GBP_SOURCE_FILE = "Publis_GBP.xlsx"



In [5]:
# FUNCIONES TRAIDAS DE GBP.PY

def gbp_get_sku_publis_____RENOMBRAR_PARA_USAR():
    """Trae del excel de publis de GBP, un listado con dicts con ID_ML y SKU principal de las publicaciones de ML en GBP

    Returns:
        _list_: Lista con diccionarios formato {'ml_id': 'MLA636710307', 'sku': '71252'}
    """

    # trae publis de GBP (publis_gbp)
    # os.chdir("C:\!PYTHON\ML")
    os.chdir(os.path.dirname(os.path.abspath(__file__)))
    workbook = load_workbook(filename="Publis_GBP.xlsx")
    sheet = workbook.active
    rows = int(sheet.dimensions[5:])

    publis_gbp = []

    for x in range(2,rows + 1):
        publi_gbp = {"ml_id": sheet["D"+str(x)].value, "sku": sheet["F"+str(x)].value}
        publis_gbp.append(copy.deepcopy(publi_gbp))

    return publis_gbp

def proveedor_del_sku_____RENOMBRAR_PARA_USAR():
    """ trae proveedor id de cada sku del excel "Articulos_GBP_extendida.xlsx" exportado de GBP

    Returns:
        _list_: Lista con diccionarios conteniendo sku, nombre proveedor y id proveedor
                Ej: {'sku': 'kit-21048-93514', 'proveedor_nombre': 'Goris Group', 'proveedor_id': '14'}
    """

    proveedores_x_skus = [] 
    os.chdir(os.path.dirname(os.path.abspath(__file__)))
    # os.chdir("C:/!PYTHON/ML")
    workbook = load_workbook(filename="Articulos_GBP_extendida.xlsx")
    sheet = workbook.active
    rows = sheet.max_row

    
    for x in range(2,rows + 1):
        proveedor_x_sku = {}
        sku = sheet["F"+str(x)].value
        proveedor_nombre = sheet["T"+str(x)].value.split(" | ", 3)[0]
        proveedor_id = sheet["T"+str(x)].value.split(" | ", 3)[1]
        proveedor_x_sku.update({"sku": sku, "proveedor_nombre": proveedor_nombre, "proveedor_id": proveedor_id})
        proveedores_x_skus.append(proveedor_x_sku)
        # print(proveedor_id, proveedor_nombre)
    
    return proveedores_x_skus



In [6]:
#   CREA LISTA CON PROVEEDORES CON COSTO DE ENVIO EN GBP
proveedores = [Proveedor(alias=a) for a in ALIAS_PROVEEDORES_CON_COSTO_ENVIO]

#def lista_coeficientes(umbrales: Dict) -> List[Coeficiente_Envio]:
#    return [Coeficiente_Envio(umbral=k, coeficiente=v) for k, v in umbrales.items()]

#   AGREGA COEFICIENTES PARA ENVIO GBP A CADA PROVEEDOR
for p in proveedores:
    p.get_coeficientes_envio()    

#ids_proveedores_con_costo_envio = [proveedor.pid for proveedor in proveedores]

#   GENERA TOKENS Y CLIENTS PARA CADA TIENDA
#   convierte lista de tiendas en diccionario con credenciales con formato {tienda: Tienda()}
tiendas_dict = {t: Tienda(name=t) for t in LISTA_TIENDAS}
tiendas = tiendas_dict



{'access_token': 'APP_USR-3820890999415663-110916-c3c898cb340e1037b3257d191ccac6bf-77581040', 'token_type': 'Bearer', 'expires_in': 21600, 'scope': 'offline_access read write', 'user_id': 77581040, 'refresh_token': 'TG-654d3a8c00ee6e000157dc3c-77581040'}
{'access_token': 'APP_USR-7170521001674920-110916-d10fd37a8ec11f35390dc2e97b22d95c-301181249', 'token_type': 'Bearer', 'expires_in': 21600, 'scope': 'offline_access read write', 'user_id': 301181249, 'refresh_token': 'TG-654d3a8d37f2160001cfe54b-301181249'}


In [7]:
#   TRAER SKU DE CADA PUBLICACION DE ML

# Trae JSON de PUBLICACION ML
#def item_details_json(item_id, store=None, token=None, client=None):
#    """Trae json con todos los atributos de una publicación"""
#
#    if not client:
#        client = httpx.Client()
#    if not token:
#        if not store:
#            print(f"Se necesita tienda o token de la publicación {item_id} para poder traer detalles.")
#            return None
#        else:
#            token = ml_aut(store, client=client)
#
#    url = f"https://api.mercadolibre.com/items/{item_id}?include_attributes=all"
#
#    headers = {
#        'Authorization': token
#        }
#
#    response = client.get(url, headers=headers)
#    j = response.json()
#    
#    if response.status_code != 200:
#            print(f"No se pudo traer listado de publicaciones de {item_id}")
#            print(response.status_code, response.text)
#            return None    
#    else:
#         return j


In [8]:


#class Item_Details:
#    
#    @staticmethod
#    def get_json(item_id, store=None, token=None, client=None):
#        """
#        Obtiene JSON con detalles de un item de MercadoLibre
#        """
#        if not item_id:
#            raise Exception("Se requiere item_id para item_details_json")
#        if not client:
#            client = httpx.Client()
#        if not token:
#            if not store:
#                raise Exception("Se requiere store o token para item_details_json")
#            else:
#                token = ml_aut(store)
#
#        url = f"https://api.mercadolibre.com/items/{item_id}?include_attributes=all"
#
#        response = client.get(url, headers={"Authorization": token})
#        return response.json()

In [9]:
#   GENERA LISTA PUBLIS_ML CON IDS

publis_ml = []
for t in tiendas.keys():
    publis_tienda = get_items_ids(store=t, token=tiendas[t].token, client=tiendas[t].client, status="active", offset=0, limit=50)
    publis_ml.extend([Publicacion(tienda=t, item_id=pt) for pt in publis_tienda])
        
len(publis_ml)

816

In [10]:
#   GENERA LISTA PUBLIS_GBP (SIN PROVEEDOR)

publis_gbp = get_publis_gbp(sku=True, proveedor=False, tienda=True, costo_envio=True, precio=True)

len(publis_gbp)

2668

In [11]:
#   formatea el string tienda de PUBLIS GBP

def format_tienda(tienda):
    split_tienda = tienda.split(" ", 1)
    return split_tienda[0].lower()


In [12]:
#   FILTRA PUBLIS GBP ACTIVAS (QUITA LAS QUE NO ESTAN EN PUBLIS_ML)
#   AGREGA SKU a PUBLIS ML

for pgbp in publis_gbp[:]:
    found = False
    pgbp.tienda = format_tienda(pgbp.tienda)
    for pml in publis_ml:
        if pgbp.item_id == pml.item_id:
            found = True
            pml.sku = pgbp.sku
            break
    if not found:
        publis_gbp.remove(pgbp)
        
len(publis_gbp)


807

In [14]:
#   AGREGA PROVEEDOR A LAS PUBLIS GBP

#os.chdir(os.path.dirname(os.path.abspath(__file__)))
# os.chdir("C:/!PYTHON/ML")
articulos_workbook = load_workbook(filename="data/Articulos_GBP_extendida.xlsx")
articulos_sheet = articulos_workbook.active

for pgbp in publis_gbp:
    pgbp.proveedor = get_proveedor_sku(pgbp.sku, Articulos_GBP_extendida_sheet=articulos_sheet)


publis_gbp[2]


Publicacion(item_id='MLA1395052477', sku='CBS220-48P-4G-AR', proveedor=Proveedor(alias='microglobal', pid='16', name='MICROGLOBAL ARGENTINA SOCIEDAD', coeficientes_envio=[Coeficiente_Envio(umbral=90000, coeficiente=1), Coeficiente_Envio(umbral=120000, coeficiente=0.5)]), tienda='tecnorium', costos_envio=Costos_Envio(ml=None, ms=None, gbp=0), precios=Precios(venta_gbp=1712452), coeficiente_envio_gbp=None)

In [15]:
#   COMPLETA DATOS DE PUBLIS ML
#   FILTRA POR PROVEEDORES CON ENVIO 
#   DESCARTA LOS LINKS DE COBRO DE MP QUE ML LISTA COMO PUBLICACIONES

not_found = []
nueva_publis_ml = []
for pml in publis_ml:
    found = False
    
    for pgbp in publis_gbp:
        if pml.item_id == pgbp.item_id:
            #print(f'pml previo a tomar datos de pgbp:\n{pml}')
            if pgbp.proveedor.alias in ALIAS_PROVEEDORES_CON_COSTO_ENVIO:
                nueva_publi = pml
                nueva_publi.sku = pgbp.sku
                nueva_publi.proveedor = pgbp.proveedor
                nueva_publi.precios = pgbp.precios
                nueva_publi.costos_envio = Costos_Envio()
                nueva_publis_ml.append(copy.deepcopy(nueva_publi))

            found= True
            break
    if not found:
        not_found.append(pml)

publis_ml = nueva_publis_ml
        
len(publis_ml)
#   PENDIENTE: ACÁ DEBERÍA CHEKEAR SI LAS NOT_FOUND (  "channels": ["mp-merchants", "mp-link"] OR "domain_id": "MLA-MERCADO_POINT_SUPPLIES")
#   PARA CONFIRMAR QUE NO DESCARTE PUBLICACIONES EN VEZ DE LINKS DE PAGO

622

In [ ]:
#   FILTRA POR SKU DE PROVEEDORES CON ENVIO
#
#for sku in proveedores_gbp[:]:
#    if sku["proveedor_id"] not in ids_proveedores_con_costo_envio:
#        proveedores_gbp.remove(sku)
#
#len(proveedores_gbp)
##ids_proveedores_con_costo_envio

In [ ]:
#   DESCARTA PUBLIS DE OTROS PROVEEDORES

#print(len(publis_ml))
#a_borrar = []
#for p in publis_ml[:]:
#    found = False
#    for s in proveedores_gbp:
#        if p.sku == s["sku"]:
#            found = True
#            p.proveedor = s["proveedor_nombre"]
#
#    if not found:
#        publis_ml.remove(p)
#
#print(len(publis_ml))
##publis_ml[1]

In [ ]:
def costos_envio_colecta_publi_json(item_id = None, token = None, client=None, item_json = None) -> Union[Dict, None]:
    """Trae json de costos promedio aproximados (el que informa ML en la interface) de envío por correo de una publicación para ML y MS.

    Args:
        item_id (_str_, optional): ID publicación. Defaults to None.
        token (_str_, optional): Defaults to None.
        item_json (_json_, optional): JSON del recurso /item de la publicación. Defaults to None.

        Si se usa item_json como argumento, no se necesitan item_id y token, y viceversa.

    Returns:
        Dict (json)
    """

    if item_json:
        j = item_json
    elif not item_id:
        print("Se requiere item_id o item_json para calcular el costo de envío")
        return None
    else:
        if not token:
            token = ml_aut(tienda_publi(item_id))

        if not client:
            client= httpx.Client()

        ZIP_CODE = "1602"
         
        url = f'https://api.mercadolibre.com/items/{item_id}/shipping_options?zip_code={ZIP_CODE}'

        headers = {
          'Authorization': token
        }

        count = 0
        while count < 6:
            response = client.get(url, headers=headers)
            j = response.json()
            if response.status_code == 404:
                #print(f'La publicación {item_id} no tiene Mercado Envíos')
                return None
            elif response.status_code != 200:
                print(f'No se pudo traer json con costo de envío de la publicación {item_id}')
                print(response.status_code, response.text)
                count += 1
                print(f'Intento {count} para la publicación {item_id}')
                if count < 6:
                    print('Reintentando...')
                else:
                    print(f'Se cancela la búsqueda de costo de envío de {item_id}')
                    return None
            else:
                if count > 0:
                    print(f'Se obtuvo json con costo de envío de la publicación {item_id} con exito al intento {count + 1}')
                return j

def costos_envio_colecta_publi(costos_envio_colecta_publi_json) -> Union[Costos_Envio, None]:

    if costos_envio_colecta_publi_json == None:
        return None
    j = costos_envio_colecta_publi_json
    costo_ml = None
    for x in j["options"]:
        if x["name"] ==  "Estándar a domicilio":   # Envío por Correo Estandar a domicilio
            costo_ml = x["list_cost"]
            break
        elif x["name"] == "Express a domicilio":
            costo_ml = x["list_cost"]
        elif x["name"] == "Prioritario a domicilio" and x["shipping_method_type"] == "next_day":
            costo_ml = x["list_cost"]
    if not costo_ml:
        print(f'La publicación {item_id} no tiene opciones de Envío por Correo Estandar, Express a Domicilio o Prioritario a Domicilio')
        return None
    else:
        costo_ms = costo_ml * 2
        costo_envio = Costos_Envio(ml=costo_ml, ms=costo_ms)
        #costos_envio = {"ml": costo_ml, "ms": costo_ms}    
        return costo_envio

def tiene_mercado_envio(item_json) -> bool:
    if item_json["shipping"]["mode"] == "not_specified" or item_json["shipping"]["mode"] == None:
        return False
    else:
        return True

In [ ]:
#   TRAE COSTOS DE ENVIO ML 
#   esto podría hacerse asincrónico para bajar el tiempo de ejecución (>3 min)

#   CONFIG COEFICIENTE CALCULO ENVIO MS RESPECTO ENVIO ML
MS_SHIPMENT_MULTIPLIER  = 2

for i, p in enumerate(publis_ml[:]):
    #   SHIPMENT_TYPES_CATALOG
    item_json = p.get_item_shipment_options_json(token=tiendas[p.tienda.lower()].token, client=tiendas[p.tienda.lower()].client)
    #print(p.tienda)
    if item_json:
        print(i, p.item_id)
        p.shipment_options = p.get_item_free_shipment_options(item_json, include_prices=True, shipment_types_catalog=SHIPMENT_TYPES_CATALOG)
        print(f'shipment options: {p.shipment_options}')
        p.get_reference_shipment_costs(ms_shipment_multiplier=MS_SHIPMENT_MULTIPLIER)
        #print(i, p.item_id, p.costos_envio)


In [ ]:
#   CALCULA COSTOS ENVIO GBP EN PUBLIS ML

for i, p in enumerate(publis_ml):
    p.set_coeficiente_envio_gbp()
    p.set_costo_envio_gbp()
    print(f'Envío GBP POST: {i} {p.item_id} {p.precios.venta_gbp} {p.coeficiente_envio_gbp} {p.costos_envio.gbp}')


In [ ]:
###   COMPARA COSTO DE ENVIO GBP ENTRE PUBLIS ML Y PUBLIS GBP
###   CREA LISTA CON PUBLIS A ACTUALIZAR

actualizacion_publis = []
for publiml in publis_ml:
    for publigbp in publis_gbp:
        if publiml.item_id == publigbp.item_id:
            if publiml.costos_envio.gbp != publigbp.costos_envio.gbp and publiml.costos_envio.gbp != None:
                actualizacion_publis.append(publiml)
                print(f'{publiml.item_id} venta {publiml.precios.venta_gbp} - envio gbp orig {publigbp.costos_envio.gbp} - envío gbp nuevo {publiml.costos_envio.gbp}')
        

In [ ]:
# CONFIG GBP_UPDATER_FILE

GBP_UPDATER_FILE = "actualizador_publis_gbp.xlsx"
GBP_UPDATER_FILE_TITLE_CELLS = {"A1": "ID de Publicación", "B1": "Costo de Envío"}
GBP_UPDATER_FILE_SHEET_TITLE = "Publicaciones"
GBP_UPDATER_FILE_COLUMNS_WIDTH = 18

In [ ]:
### ESCRIBE EXCEL PARA IMPORTAR EN GBP CON PUBLICACIONES A ACTUALIZAR

# WORKBOOK REQUERIMENTS
# Ingresar importe como TEXTO, con el separador decimal como COMA (ej: "1599,00")

def save_to_excel(publis_para_actualizar: List[Publicacion], excel_pathname: str, gbp_updater_file_title_cells: Dict[str, str], \
                  gbp_updater_file_sheet_title: str = "Publicaciones", gbp_updater_file_columns_width: int = 18) -> bool:
    """Genera archivo excel con publicaciones con costo de envío a actualizar en GBP para importar en el sistema. 
    Cumple con la versión 17 del template que entrega GBP. Siempre confirmar que el formato actual siga siendo compatible.
    Se puede acceder a la versión actual desde acá:
    http://gbp47.globalbluepoint.com/tecnorium/App_HelpFiles/Cambio%20masivo%20de%20Estados%20de%20Publicaciones.xlsx
     
    Args:
        publis_para_actualizar (List[Publi_gbp]): Listado de publicaciones (con objeto Publi_gbp) a actualizar.
        excel_path (str): Path y nombre del archivo a generar.
        gbp_updater_file_title_cells (Dict[str, str]): Columnas del excel a generar. Chekear en GBP que no hayan cambiado.
        gbp_updater_file_sheet_title (str): Nombre de la hoja del excel. Defaults to "Publicaciones". Chekear en GBP que no haya cambiado el nombre requerido.
        gbp_updater_file_columns_width (int): Ancoh de las columnas. No afecta el funcionamiento pero facilita el control del excel si necesitamos abrirlo. Defaults to 15.

    Returns:
        bool: Si está todo bien, devuelve True. Si falta algún argumento o alguno tiene el tipado incorrecto, devuelve False.
    """

    # Checks if all the arguments are present and correct 
    if not publis_para_actualizar or not excel_pathname or not gbp_updater_file_title_cells or not gbp_updater_file_sheet_title or not gbp_updater_file_columns_width:
         print('Falta un argumento.\nUso: save_to_excel(publis_para_actualizar: List[Publi_gbp], excel_pathname: str, gbp_updater_file_title_cells: Dict[str, str], \
                  gbp_updater_file_sheet_title: str = "Publicaciones", gbp_updater_file_columns_width: int = 15)')
         return False
    elif type(publis_para_actualizar ) != list:
         return False
    elif type(excel_pathname) != str:
         return False
    elif type(gbp_updater_file_title_cells) != dict:
         return False
    elif type(gbp_updater_file_sheet_title) != str:
         return False
    elif type(gbp_updater_file_columns_width) != int:
         return False
    else:
        # Create a new workbook
        gbp_updater_wb = Workbook()
        gbp_updater_sheet = gbp_updater_wb.active
        gbp_updater_sheet.title = gbp_updater_file_sheet_title
        col_letters = [col.strip("1") for col in gbp_updater_file_title_cells.keys()]

        # Set columns width (just for OCT)
        for col in col_letters:
            gbp_updater_sheet.column_dimensions[col].width = gbp_updater_file_columns_width

        # Set workbook column names
        for column_cell in gbp_updater_file_title_cells.keys():
            gbp_updater_sheet[column_cell].value = gbp_updater_file_title_cells[column_cell]

        # Write the new content
        for row in range(2, len(publis_para_actualizar) + 2):
            gbp_updater_sheet[f'A{row}'] = publis_para_actualizar[row - 2].item_id
            gbp_updater_sheet[f'B{row}'] = f'{str(publis_para_actualizar[row - 2].costos_envio.gbp)},00'


        # Saves the excel file over existing one
        gbp_updater_wb.save(filename=excel_pathname)
        gbp_updater_wb.close()
        return True


save_to_excel(publis_para_actualizar = actualizacion_publis, excel_pathname = GBP_UPDATER_FILE, \
              gbp_updater_file_title_cells = GBP_UPDATER_FILE_TITLE_CELLS, gbp_updater_file_sheet_title = GBP_UPDATER_FILE_SHEET_TITLE, \
                gbp_updater_file_columns_width = GBP_UPDATER_FILE_COLUMNS_WIDTH)